In [3]:
# let's take one dataset for example
from multilab.datasets import reuter
sentences, labels = reuter()

from multilab.preprocess import Text_preprocessing
text_preprocessing = Text_preprocessing()

dataframe = text_preprocessing.labels_to_dataframe(sentences,labels)
preprocessded_dataset = text_preprocessing.initial_preprocess(dataframe, chunk_value = 25)
dataset, frequency_list = text_preprocessing.keep_labels(preprocessded_dataset,keep_ratio=0.10)
slice_dataset = text_preprocessing.dataset_slice(dataset,ratio=0.25)

import numpy as np
all_sente = list(slice_dataset['text'])
all_label = np.array(slice_dataset.drop('text', 1))

X_train, X_test, y_train, y_test = text_preprocessing.split_dataset(all_sente, all_label)

[nltk_data] Downloading package reuters to /Users/monk/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
lower_case done
punctuation removed
text cleaning done


In [4]:
from multilab.models import Elmo

config = {
                         'no_of_labels'               : len(y_train[0]),
                         'learning_rate'              : 0.001,
                         'epoch'                      : 2,
                         'batch_size'                 : 128,
                         'result_path'                : '/Users/monk/Desktop'
                        }


el_m = Elmo(X_train, y_train, X_test,  y_test, config)

In [5]:
print(el_m.train())

epoch 0,  iteration 11,  F1_score 0.5370370370370371,  loss 3.2547619342803955: 100%|██████████| 12/12 [03:21<00:00, 15.89s/it]
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
Bar desc:   0%|          | 0/12 [00:00<?, ?it/s]

validation_acc {'subset_accuracy': 0.42333333333333334, 'hamming_score': 0.4255555555555555, 'hamming_loss': 0.08925925925925926, 'micro_ac': 0.5157070000747034, 'weight_ac': 0.35893647346555246, 'epoch': 0}


epoch 1,  iteration 0,  F1_score 0.4476190476190476,  loss 3.137950897216797:   8%|▊         | 1/12 [00:20<03:42, 20.22s/it]

KeyboardInterrupt: 